# Lab: Deploy agents on Amazon Bedrock AgentCore

This lab is modified from the Lab3 of the original sample.


![architecture](./images/temporalarchitecture.png)


## What You Will Learn

You'll learn how to leverage AgentCore's purpose-built infrastructure for running agents at scale while maintaining security, performance, and reliability standards required for enterprise applications.

## Amazon Bedrock AgentCore Runtime

[Amazon Bedrock AgentCore Runtime](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/agents-tools-runtime.html) provides a secure, serverless, and purpose-built hosting environment for deploying and running AI agents or tools, shortening the time to value from experiments to production-grade agents.

[Learn more about how AgentCore Runtime works](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/runtime-how-it-works.html)

## Amazon Bedrock AgentCore Observability

[AgentCore Observability](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/observability.html) helps you trace, debug, and monitor agent performance in production environments. It offers detailed visualizations of each step in the agent workflow, enabling you to inspect an agent's execution path, audit intermediate outputs, and debug performance bottlenecks and failures.

### Step 1: Install dependencies and aws configs

In [ ]:
# Install dependencies for AgentCore deployment
!pip install --force-reinstall -U -r requirements.txt --quiet

In [ ]:
!aws configure set sso_start_url https://temporal.awsapps.com/start --profile corp-sso
!aws configure set sso_region us-west-2 --profile corp-sso
!aws configure set sso_account_id 269172689222 --profile corp-sso
!aws configure set sso_role_name AWSAdministratorAccess --profile corp-sso
!aws configure set region us-west-2 --profile corp-sso
!aws configure set output json --profile corp-sso

In [ ]:
!aws sso login --profile corp-sso

In [ ]:
import os, boto3
os.environ["AWS_PROFILE"]="corp-sso"
os.environ["AWS_REGION"]="us-west-2"          # your service region
os.environ["AWS_DEFAULT_REGION"]="us-west-2"
os.environ["AWS_SDK_LOAD_CONFIG"]="1"
boto3.Session().client("sts").get_caller_identity()

In [ ]:
# Import AgentCore Runtime deployment tools and utilities
import uuid
from utils import setup_cognito_user_pool, reauthenticate_user
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
from typing import Any, Optional
import urllib.parse
import requests
import json

In [ ]:
# Initialize AWS session and get current region
boto_session = Session()
region = boto_session.region_name
print(region)

### Step 2: Setting up Amazon Cognito for Authentication

AgentCore Runtime requires authentication. We'll use Amazon Cognito to provide JWT tokens for accessing our deployed agent server.

In [ ]:
# Reload the utils.agentcore_utils submodule to pick up changes
# I've reorded the above cells so that this shouldn't be needed in the future.
import importlib
from utils import agentcore_utils
importlib.reload(agentcore_utils)

# Also reload utils to refresh the imports
import utils
importlib.reload(utils)

# Re-import the function
from utils import setup_cognito_user_pool, reauthenticate_user

print("Module reloaded successfully")

In [ ]:
# Set up Amazon Cognito for AgentCore Runtime authentication
print("Setting up Amazon Cognito user pool...")

cognito_config = setup_cognito_user_pool()

print("Cognito setup completed ✓")
print(f"User Pool ID: {cognito_config.get('user_pool_id', 'N/A')}")
print(f"Client ID: {cognito_config.get('client_id', 'N/A')}")

In [ ]:
# Configure JWT authorization for AgentCore Runtime
auth_config = {
    "customJWTAuthorizer": {
        "allowedClients": [cognito_config["client_id"]],
        "discoveryUrl": cognito_config["discovery_url"],
    }
}

print(auth_config)

### Step 4.2: Launching agent to AgentCore Runtime

Now that we have a Dockerfile, let's launch the agent to the AgentCore Runtime. This will create the Amazon ECR repository and the AgentCore Runtime.

In [ ]:
# Deploy agent to AgentCore Runtime (creates ECR repo and runtime)
print("Launching Agent server to AgentCore Runtime...")
print("This may take several minutes...")

launch_result = agentcore_runtime.launch(
    env_vars={"OTEL_PYTHON_EXCLUDED_URLS": "/ping,/invocations"}
)

print("Launch completed ✓")
print(f"Agent ARN: {launch_result.agent_arn}")
print(f"Agent ID: {launch_result.agent_id}")

### Step 5: Invoking AgentCore Runtime

Finally, we can invoke our AgentCore Runtime with a payload

In [ ]:
# Authenticate user and get bearer token for API access
bearer_token = reauthenticate_user(client_id=cognito_config["client_id"])

In [ ]:
def invoke_endpoint(
    agent_arn: str,
    payload,
    session_id: str,
    bearer_token: Optional[str],
    region: str = "us-west-2",
    endpoint_name: str = "DEFAULT",
) -> Any:
    """Invoke agent endpoint using HTTP request with bearer token."""
    escaped_arn = urllib.parse.quote(agent_arn, safe="")
    url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{escaped_arn}/invocations"
    headers = {
        "Authorization": f"Bearer {bearer_token}",
        "Content-Type": "application/json",
        "X-Amzn-Bedrock-AgentCore-Runtime-Session-Id": session_id,
    }

    try:
        body = json.loads(payload) if isinstance(payload, str) else payload
    except json.JSONDecodeError:
        body = {"payload": payload}

    try:
        response = requests.post(
            url,
            params={"qualifier": endpoint_name},
            headers=headers,
            json=body,
            timeout=100,
            stream=True,
        )
        last_data = False
        for line in response.iter_lines(chunk_size=1):
            if line:
                line = line.decode("utf-8")
                if line.startswith("data: "):
                    last_data = True
                    line = line[6:]
                    line = line.replace('"', "")
                    yield line
                elif line:
                    line = line.replace('"', "")
                    if last_data:
                        yield "\n" + line
                    last_data = False

    except requests.exceptions.RequestException as e:
        print("Failed to invoke agent endpoint: %s", str(e))
        raise

In [ ]:
for chunk in invoke_endpoint(
    agent_arn=launch_result.agent_arn,
    payload={
        "prompt": "I make $6000/month. Help me create a budget and suggest an investment portfolio."
    },
    session_id=str(uuid.uuid4()),
    bearer_token=bearer_token,
):
    print(chunk.replace("\\n", "\n"), end="")


In [ ]:
def invoke_endpoint_non_streaming(
    agent_arn: str,
    payload,
    session_id: str,
    bearer_token: Optional[str],
    region: str = "us-west-2",
    endpoint_name: str = "DEFAULT",
) -> Any:
    """Invoke agent endpoint using HTTP request with bearer token."""
    escaped_arn = urllib.parse.quote(agent_arn, safe="")
    url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{escaped_arn}/invocations"
    headers = {
        "Authorization": f"Bearer {bearer_token}",
        "Content-Type": "application/json",
        "X-Amzn-Bedrock-AgentCore-Runtime-Session-Id": session_id,
    }

    try:
        body = json.loads(payload) if isinstance(payload, str) else payload
    except json.JSONDecodeError:
        body = {"payload": payload}

    try:
        response = requests.post(
            url,
            params={"qualifier": endpoint_name},
            headers=headers,
            json=body,
            timeout=100,
        )
        return response.text

    except requests.exceptions.RequestException as e:
        print("Failed to invoke agent endpoint: %s", str(e))
        raise

In [ ]:
response = invoke_endpoint_non_streaming(
    agent_arn=launch_result.agent_arn,
    payload={
        "prompt": "I make $6000/month. Help me create a budget and suggest an investment portfolio."
    },
    session_id=str(uuid.uuid4()),
    bearer_token=bearer_token,
)
print(response)

## Temporal

### Step 3: Deploying the agent to AgentCore Runtime

The `CreateAgentRuntime` operation supports comprehensive configuration options, letting you specify container images, environment variables, and encryption settings. You can also configure protocol settings (HTTP, MCP) and authorization mechanisms to control how your clients communicate with the agent. 

**Note:** Operations best practice is to package code as a container and push to ECR using CI/CD pipelines and IaC.

In this tutorial, we will use the Amazon Bedrock AgentCore Python SDK to easily package your artifacts and deploy them to AgentCore Runtime.

### Step 3.1: Configure AgentCore Runtime deployment

First, we will use our starter toolkit to configure the AgentCore Runtime deployment with an entrypoint, the execution role we just created, and a requirements file. We will also configure the starter kit to auto-create the Amazon ECR repository on launch.

During the configure step, your Dockerfile will be generated based on your application code.

![runtime](./images/runtime_overview.png)

In [ ]:
# Configure AgentCore Runtime deployment settings
agentcore_runtime_temporal = Runtime()

agent_name_temporal = "temporal_orchestrated_financial_assistant"

print("Configuring AgentCore Runtime...")
response = agentcore_runtime_temporal.configure(
    entrypoint="worker.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name_temporal,
    authorizer_configuration=auth_config,
)

print("Configuration completed ✓")

Set Temporal connectivity configs into env vars

In [ ]:
from dotenv import load_dotenv

load_dotenv(override=True)

os.environ["TEMPORAL_ADDRESS"]="us-east-1.aws.api.temporal.io:7233"
os.environ["TEMPORAL_NAMESPACE"]="cornelia3.a2dd6"
# os.environ["TEMPORAL_API_KEY"]= in .env
session_id = "313d4e11-34d5-40bd-a5e9-82e6911ddcb8" 

### Step 3.2: Launching agent to AgentCore Runtime

Now that we have a Dockerfile, let's launch the agent to the AgentCore Runtime. This will create the Amazon ECR repository and the AgentCore Runtime.

In [ ]:
# Deploy agent to AgentCore Runtime (creates ECR repo and runtime)
print("Launching Temporal Worker to AgentCore Runtime...")
print("This may take several minutes...")

temporal_address = os.getenv("TEMPORAL_ADDRESS", "us-east-1.aws.api.temporal.io:7233")
temporal_namespace = os.getenv("TEMPORAL_NAMESPACE", "default")
temporal_api_key = os.getenv("TEMPORAL_API_KEY") 

launch_result_temporal = agentcore_runtime_temporal.launch(
    env_vars={"OTEL_PYTHON_EXCLUDED_URLS": "/ping,/invocations", "TEMPORAL_ADDRESS": temporal_address, "TEMPORAL_NAMESPACE": temporal_namespace, "TEMPORAL_API_KEY": temporal_api_key}
)

print("Launch completed ✓")
print(f"Agent ARN: {launch_result_temporal.agent_arn}")
print(f"Agent ID: {launch_result_temporal.agent_id}")

### Update runtime settings

The following code block will allow you to set a new container idle timeout. The default is 15 mins but for demo purposes, where we want to show the lifecycle events of the container, we want to shorten it.

In [ ]:
import boto3
import json
import os

# -------------------------
# CONFIGURE HERE
REGION = os.getenv("AWS_REGION", "us-west-2")
NEW_IDLE_TIMEOUT = 60   # seconds.
# -------------------------

control = boto3.client("bedrock-agentcore-control", region_name=REGION)

# ---- Step 1: List runtimes ----
print("📋 Available AgentCore runtimes:\n")

runtimes = []
next_token = None

while True:
    kwargs = {}
    if next_token:
        kwargs["nextToken"] = next_token

    resp = control.list_agent_runtimes(**kwargs)
    runtimes.extend(resp.get("agentRuntimes", []))
    next_token = resp.get("nextToken")
    if not next_token:
        break

if not runtimes:
    print("⚠️ No runtimes found in this region.")
    raise SystemExit

for i, rt in enumerate(runtimes, start=1):
    print(f"{i}. name={rt.get('name')}  id={rt.get('agentRuntimeId')}  arn={rt.get('agentRuntimeArn')}")

choice = input("\nEnter the NAME or ID of the runtime to update: ").strip()

# ---- Step 2: Select the runtime ----
selected = None
for rt in runtimes:
    if choice == rt.get("name") or choice == rt.get("agentRuntimeId"):
        selected = rt
        break

if not selected:
    print(f"❌ Runtime '{choice}' not found.")
    raise SystemExit

runtime_id = selected["agentRuntimeId"]
print(f"\n✔ Selected runtime:\n  name={selected.get('name')}\n  id={runtime_id}\n")

# ---- Step 3: Fetch current config ----
resp = control.get_agent_runtime(agentRuntimeId=runtime_id)

# Some SDK versions nest config under "agentRuntime"
runtime_cfg = resp.get("agentRuntime", resp)

print("Current lifecycleConfiguration:")
print(json.dumps(runtime_cfg.get("lifecycleConfiguration", {}), indent=2))

current_lifecycle = runtime_cfg.get("lifecycleConfiguration", {})
new_lifecycle = {
    "idleRuntimeSessionTimeout": NEW_IDLE_TIMEOUT,
    "maxLifetime": current_lifecycle.get("maxLifetime", 28800),
}

print("\n🔧 Updated lifecycleConfiguration will be:")
print(json.dumps(new_lifecycle, indent=2))

confirm = input("\nProceed with update? [y/N]: ").strip().lower()
if confirm == "y":

    # ---- Step 4: Update the runtime ----
    # Build update parameters, only including optional fields if they have valid values
    update_params = {
        "agentRuntimeId": runtime_id,
        "agentRuntimeArtifact": runtime_cfg["agentRuntimeArtifact"],
        "roleArn": runtime_cfg["roleArn"],
        "networkConfiguration": runtime_cfg["networkConfiguration"],
        "protocolConfiguration": runtime_cfg["protocolConfiguration"],
        "lifecycleConfiguration": new_lifecycle,
        "environmentVariables": runtime_cfg.get("environmentVariables", {}),
    }

    # Only include description if it exists and is not empty (API requires min length 1)
    description = runtime_cfg.get("description", "")
    if description:
        update_params["description"] = description

    # Only include authorizerConfiguration if it exists and is not empty
    authorizer_config = runtime_cfg.get("authorizerConfiguration", {})
    if authorizer_config:
        update_params["authorizerConfiguration"] = authorizer_config

    # Only include requestHeaderConfiguration if it exists and is not empty
    # (API requires at least one key to be set, e.g., requestHeaderAllowlist)
    request_header_config = runtime_cfg.get("requestHeaderConfiguration", {})
    if request_header_config:
        update_params["requestHeaderConfiguration"] = request_header_config

    update_resp = control.update_agent_runtime(**update_params)

    print("\n✅ Runtime updated successfully.\nResponse:")

else:
    print("🚫 Update cancelled but reporting current config")
    update_resp = control.get_agent_runtime(agentRuntimeId=runtime_id)

# Convert response to JSON-serializable format (handle datetime objects)
def json_serial(obj):
    """JSON serializer for objects not serializable by default json code"""
    from datetime import datetime, date
    if isinstance(obj, (datetime, date)):
        return obj.isoformat()
    raise TypeError(f"Type {type(obj)} not serializable")

print(json.dumps(update_resp, indent=2, default=json_serial))


### Step 4: Invoking AgentCore Runtime

There are a number of ways that the worker container gets shut down. Invoking the endpoint will either reset the idle timeout (if still running) or start up a new instance (container)

In [ ]:
# Authenticate user and get bearer token for API access
bearer_token = reauthenticate_user(client_id=cognito_config["client_id"])

In [ ]:
def invoke_endpoint_worker(
    agent_arn: str,
    payload,
    session_id: str,
    bearer_token: Optional[str],
    region: str = "us-west-2",
    endpoint_name: str = "DEFAULT",
) -> Any:
    """Invoke agent endpoint using HTTP request with bearer token."""
    escaped_arn = urllib.parse.quote(agent_arn, safe="")
    url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{escaped_arn}/invocations"
    headers = {
        "Authorization": f"Bearer {bearer_token}",
        "Content-Type": "application/json",
        "X-Amzn-Bedrock-AgentCore-Runtime-Session-Id": session_id,
    }

    try:
        body = json.loads(payload) if isinstance(payload, str) else payload
    except json.JSONDecodeError:
        body = {"payload": payload}

    try:
        response = requests.post(
            url,
            params={"qualifier": endpoint_name},
            headers=headers,
            json=body,
            timeout=100,
        )
        return response.text

    except requests.exceptions.RequestException as e:
        print("Failed to invoke agent endpoint: %s", str(e))
        raise

In [ ]:
response = invoke_endpoint_worker(
    agent_arn=launch_result_temporal.agent_arn,
    payload={
        "prompt": "Noop - keep alive hack"
    },
    session_id=str(session_id),
    bearer_token=bearer_token,
)
print(f"session_id: {session_id}")
print(response)

## Cleanup (Optional)

Let's now clean up the AgentCore Runtime resources created.

In [ ]:
## Optional: Clean up guardrail (commented out)
# delete_guardrail()

In [ ]:
## Optional: Clean up Cognito user pool (commented out)
# delete_cognito_user_pool()

In [ ]:
## Get deployment details for cleanup (commented out)
# launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split("/")[1]

In [ ]:
## Optional: Delete AgentCore Runtime and ECR repository (commented out)
# agentcore_control_client = boto3.client("bedrock-agentcore-control", region_name=region)
# ecr_client = boto3.client("ecr", region_name=region)

# runtime_delete_response = agentcore_control_client.delete_agent_runtime(
#     agentRuntimeId=launch_result.agent_id,
# )

# response = ecr_client.delete_repository(
#     repositoryName=launch_result.ecr_uri.split("/")[1], force=True
# )